# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [ ]:
import pandas as pd

## imports for Python, Pandas

In [ ]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [ ]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [ ]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

In [ ]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [ ]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [ ]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [2]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

#1. The 10 countries with the most projects:

#load the json data into a dataframe: world_bank_projects_df
world_bank_projects_df = pd.read_json('data/world_bank_projects.json')

#create mostProjects dataframe consisting of top 10 countries based on number of times a country is listed in world_bank_projects_df
mostProjects = world_bank_projects_df.groupby(['countrycode', 'countryname']).agg('count').nlargest(10, '_id')
print(mostProjects)

                                             _id  approvalfy  \
countrycode countryname                                        
CN          People's Republic of China        19          19   
ID          Republic of Indonesia             19          19   
VN          Socialist Republic of Vietnam     17          17   
IN          Republic of India                 16          16   
RY          Republic of Yemen                 13          13   
BD          People's Republic of Bangladesh   12          12   
MA          Kingdom of Morocco                12          12   
NP          Nepal                             12          12   
3A          Africa                            11          11   
MZ          Republic of Mozambique            11          11   

                                             board_approval_month  \
countrycode countryname                                             
CN          People's Republic of China                         19   
ID          Republic of 

In [3]:
#2. Find the top 10 major project themes (using column 'mjtheme_namecode')

#First, look at the top 10 major project themes by project code:
#load json into string world_bank_projects_string, then create dataframe
world_bank_projects_string = json.load((open('data/world_bank_projects.json')))

project_themes_codes = json_normalize(world_bank_projects_string, 'mjtheme_namecode')

#review the number of unique codes and names
project_themes_codes_unique = project_themes_codes.groupby('code').nunique()
print("Unique Values: \n", project_themes_codes_unique)

#groupby and count number of times code is used in the world_bank_projects, store in project_themes_code dataframe
project_themes_codes = project_themes_codes.groupby('code').agg('count').nlargest(10, 'name')

print(project_themes_codes)


Unique Values: 
       code  name
code            
1        1     2
10       1     2
11       1     2
2        1     2
3        1     2
4        1     2
5        1     2
6        1     2
7        1     2
8        1     2
9        1     2

      name
code      
11     250
10     216
8      210
2      199
6      168
4      146
7      130
5       77
9       50
1       38


In [13]:
#Now, look at the top 10 major theme by project name:
#create dataframe of theme name and codes
project_themes_names = json_normalize(world_bank_projects_string, 'mjtheme_namecode')

#groupby and count number of times name is used in the world_bank_projects, store in project_themes_names dataframe
project_themes_names = project_themes_names.groupby('name').agg('count').nlargest(10, 'code')

print(project_themes_names)

                                              code
name                                              
Environment and natural resources management   223
Rural development                              202
Human development                              197
Public sector governance                       184
Social protection and risk management          158
Financial and private sector development       130
                                               122
Social dev/gender/inclusion                    119
Trade and integration                           72
Urban development                               47


In [23]:
#Based on the top 10 by code and top 10 by name and the unique values for codes and names, it is clear there 
#are both blank and non-blank name values associated with the codes.  
#To resolve this, create a lookup table (dataframe) that can be used to associate codes and names, prior 
#to finding the top 10 major themes

#create dataframe of mjtheme_namecode
project_themes = json_normalize(world_bank_projects_string, 'mjtheme_namecode')

#remove the duplicates and sort alphabetically by name
project_themes = project_themes.drop_duplicates().sort_values(by='name')

#replace blank names with np.nan
project_themes['name'].replace('', np.nan, inplace=True)

#drop rows where name column contains nan
project_themes.dropna(subset=['name'], inplace=True)

#reset index of the new dataframe/lookup table
project_themes.reset_index(drop=True, inplace=True)

print(project_themes)

   code                                          name
0     1                           Economic management
1    11  Environment and natural resources management
2     4      Financial and private sector development
3     8                             Human development
4     2                      Public sector governance
5     3                                   Rule of law
6    10                             Rural development
7     7                   Social dev/gender/inclusion
8     6         Social protection and risk management
9     5                         Trade and integration
10    9                             Urban development


In [64]:
#now, recreate the dataframe of top 10 themes, using the lookup table:

#recreate project_themes_names dataframe from the nested mjtheme_namecode
project_themes_names = json_normalize(world_bank_projects_string, 'mjtheme_namecode')

#merge project_themes_names dataframe with the project_themes lookup table to get codes and names matched up
top_10_themes = pd.merge(project_themes_names, project_themes, how='inner', on=['code', 'code'])

#keep all codes and names only from the lookup dataframe
top_10_themes = top_10_themes[['code', 'name_y']]

#groupby name_y and sort descending by count, rename columns
top_10_themes = top_10_themes.groupby(['name_y']).size().sort_values(ascending=False).reset_index()
top_10_themes.columns = ['name', 'count']

#keep only the top 10 themes
top_10_themes = top_10_themes.nlargest(10, 'count')

print(top_10_themes)

                                           name  count
0  Environment and natural resources management    250
1                             Rural development    216
2                             Human development    210
3                      Public sector governance    199
4         Social protection and risk management    168
5      Financial and private sector development    146
6                   Social dev/gender/inclusion    130
7                         Trade and integration     77
8                             Urban development     50
9                           Economic management     38
